In [4]:
import glob
import os
import csv
import pandas as pd
from torchvision import transforms

In [ ]:
def mean_normalization(image, minimum, maximum):
    return ((image- minimum) / (maximum - minimum))

class MyFitsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.row = pd.read_csv(csv_file) #image name and label is in csv
        self.root_dir = root_dir
        self.transform = transform
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.max_w = 2862
        self.max_h = 3361
        self.mean = 0.19110974568257244
        self.min = -1500
        self.max = 1500

    def __getitem__(self, index):
        filename = self.row.iloc[index, 0]
        foldername = filename.split(".")[2]
        img_path = os.path.join(self.root_dir, foldername, filename)
        hmi = sunpy.map.Map(img_path)
        data = hmi.data.astype('float32')
        
        #nan to 0
        mask = [data != data]
        if np.sum(np.isnan(data)) > 0:
            data[mask] = 0
            
        #clip
        data = np.clip(data, -1500,1500)
        
        #mean
        data = mean_normalization(image, self.min, self.max)
        
        #Padding to 0 
        image = transforms.functional.pad(data, [self.max_h, self.max_w], fill = 0, padding_mode=0)
                
        input_image  = image
        y_label = torch.tensor(int(self.row.iloc[index, 1]))
        
        return (input_image, y_label)

        #return samples
    def __len__(self):
        return len(self.row)

